## Trip Itinerary Planner using Vertex LLM

Details about the project: [here](https://docs.google.com/document/d/14GPoh6UYXgi-7PPAXPwS5hii63phzs6BlKiu-6QnmMQ/edit?resourcekey=0-DEPXV6CQflQuJAHSa38efQ)


Use case problem
Planning a trip can be a daunting task, especially if you're trying to do it for a large group of people. There are so many things to keep track of, from flights and hotels to activities and transportation. It's easy to forget something important or make a mistake that could ruin your trip.

Use case solution
Our trip itinerary planner solves this problem by providing a simple, easy-to-use interface for planning your trip. You can enter all of the important details, such as your travel days, budget, and interests, and the planner will generate a personalized itinerary for you considering your budget. The itinerary will include everything you need to know, from where to stay to what to do each day.


Approach
Vertex AI offering to be used: Gen AI Studio ( Language) - General tips and advice
APIs
Experiments using Vertex AI Get AI Studio UI
Experiments using Vertex AI SDK for Python in Colab

Prompt Design
Below prompts have been used to get responses from LLM.

```
You are an experienced trip itinerary planner and you are working for a famous travel agency.
-Instruction:
Don't give incomplete sentences.
Don't give long descriptions

- Prepare Itinerary for {number_of_people} people traveling to {trip_destination} for {trip_days} days.
- Consider {trip_price_category} {currency} budget for the complete trip.
- Include the cost of each activity in the itinerary.
- Include a link to the photos for each day.
- Exclude information about tourist places.

```

Risks
Results depend on the performance of Gen AI LLM APIs.
The end user will not have access to the prompt sent to the model.


### The responce from the model:

Here is a sample itinerary for a 5-day trip to Bali for 2 people on a medium budget:
**Day 1**

* Arrive at Ngurah Rai International Airport (DPS)
* Check in at Hotel Puri Nusa Indah ($30/night)
* Visit Uluwatu Temple for sunset (~$5 entrance fee)
* Photos: https://bit.ly/bali-day1-photos

**Day 2**

* Hike Mount Batur ($65/person for guided tour)
* Visit Tegalalang Rice Terrace (~$10 entrance fee)
* Photos: https://bit.ly/bali-day2-photos

**Day 3**

* Take a cooking class ($40/person)
* Visit the Sacred Monkey Forest Sanctuary (~$15 entrance fee)
* Photos: https://bit.ly/bali-day3-photos

**Day 4**

* Snorkel or dive in Nusa Penida ($45/person for boat trip)
* Visit Crystal Bay Beach
* Photos: https://bit.ly/bali-day4-photos

**Day 5**

* Visit Jimbaran Beach for a seafood dinner (~$20/person)
* Depart from DPS
* Photos: https://bit.ly/bali-day5-photos

**Estimated Total Cost:** $450-500 per person

In [1]:
## Authenticate yourself
# from google.colab import auth as google_auth
# google_auth.authenticate_user()

In [2]:
## Pip install aiplatform
!pip install -q google-cloud-aiplatform

In [3]:
# GCP_PROJECT= PROJECT_ID=project_id= !(gcloud config get-value core/project)
PROJECT_ID= !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
GCP_PROJECT = PROJECT_ID
VERTEX_API_LOCATION = LOCATION = REGION = 'asia-southeast1'

print(GCP_PROJECT)
print(PROJECT_ID)

BUCKET_NAME = 'my-project-0004-bucket' # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output with gs:// prefix.
BUCKET_URI = "gs://my-project-0004-bucket"  # @param {type:"string"}


my-project-0004-346516
my-project-0004-346516


In [4]:
## Load all required libraries
from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project=PROJECT_ID, location=LOCATION)
parameters = {
    "temperature": 0,
    "max_output_tokens": 1024,
    "top_p": 0.8,
    "top_k": 40
}


In [10]:
## Get Trip details from Customer
customer_source_loc="New York, USA"     # @param {type: "string"}
trip_destination="Bali, Indonesia"        # @param {type: "string"}
currency = 'USD'
trip_price_category="Medium"            # @param {type: "string"}
number_of_people = 2
trip_days = 5

In [11]:
## Design Prompt for LLM
trip_itinery_query = '''You are an experienced trip itinerary planner and you are working for a famous travel agency.
-Instruction:
Don't give incomplete sentences.
Don't give long descriptions

- Prepare Itinerary for {number_of_people} people traveling to {trip_destination} for {trip_days} days.
- Consider {trip_price_category} {currency} budget for the complete trip.
- Include the cost of each activity in the itinerary.
- Include a link to the photos for each day.
- Exclude information about tourist places.
'''.format(customer_source_loc=customer_source_loc, number_of_people=number_of_people, trip_destination=trip_destination, trip_price_category=trip_price_category, trip_days=trip_days, currency=currency)

# - Include famous as well as virgin places to visit with the travel cost from hotel.
# - Include places where I can have breakfast, lunch and dinner.

In [12]:
print(trip_itinery_query)

You are an experienced trip itinerary planner and you are working for a famous travel agency.
-Instruction:
Don't give incomplete sentences.
Don't give long descriptions

- Prepare Itinerary for 2 people traveling to Bali, Indonesia for 5 days.
- Consider Medium USD budget for the complete trip.
- Include the cost of each activity in the itinerary.
- Include a link to the photos for each day.
- Exclude information about tourist places.



In [13]:
# ## Get the prediction from LLM
# predict_large_language_model_sample("us-central1-aiplatform.googleapis.com", "cloud-large-language-models", endpoint_id, trip_itinery_query , temperature, max_decode_steps, top_p, top_k, "us-central1")

In [15]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

def generate():
  vertexai.init(project=PROJECT_ID, location=LOCATION)
  model = GenerativeModel("gemini-1.0-pro-001")
  responses = model.generate_content(
    trip_itinery_query,
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.9,
        "top_p": 1
    },
    safety_settings={
          generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    },
    stream=True,
  )
  
  for response in responses:
    print(response.text, end="")


generate()

**Day 1**

* Arrive at Ngurah Rai International Airport (DPS)
* Check in at Hotel Puri Nusa Indah ($30/night)
* Visit Uluwatu Temple for sunset (~$5 entrance fee)
* Photos: https://bit.ly/bali-day1-photos

**Day 2**

* Hike Mount Batur ($65/person for guided tour)
* Visit Tegalalang Rice Terrace (~$10 entrance fee)
* Photos: https://bit.ly/bali-day2-photos

**Day 3**

* Take a cooking class ($40/person)
* Visit the Sacred Monkey Forest Sanctuary (~$15 entrance fee)
* Photos: https://bit.ly/bali-day3-photos

**Day 4**

* Snorkel or dive in Nusa Penida ($45/person for boat trip)
* Visit Crystal Bay Beach
* Photos: https://bit.ly/bali-day4-photos

**Day 5**

* Visit Jimbaran Beach for a seafood dinner (~$20/person)
* Depart from DPS
* Photos: https://bit.ly/bali-day5-photos

**Estimated Total Cost:** $450-500 per person